## Setup

### Imports

In [ ]:
import os
import shutil

### Update demo directories

In [ ]:
# ASTRA-sim 
astra_sim_dir = '/home/jinsun/jinsun-astra-sim/astra-sim' # Location of the astra-sim directory
os.chdir(astra_sim_dir)
os.system('git checkout collectiveapi')

# Chakra to run workload
chakra_dir = astra_sim_dir + '/extern/graph_frontend/chakra'
os.chdir(chakra_dir) # Cloned from the main chakra repo
os.system('git checkout 7a5faa') # Default submodule commit as of 2024-Jun-05
os.system('sed -i \'s/this->is_cpu_op_ = true;/this->is_cpu_op_ = false;/\' et_feeder/et_feeder_node.cpp') # Hotfix

# Chakra for msccl converter
converter_dir = '/home/jinsun/chakra/'
os.chdir(converter_dir) # Cloned from https://github.com/jinsun-yoo/chakra, NOT the main chakra repo
os.system('git checkout mscclang_converter')

# MSCCLang directory
mscclang_dir = '/home/jinsun/msccl-tools/examples/mscclang'

# Scratchpad directory for intermediate outputs
demodir = astra_sim_dir + '/demo'
if not os.path.exists(demodir): 
    os.mkdir(demodir)

### Build binaries

In [ ]:
os.chdir(astra_sim_dir)
os.system('bash build/astra_analytical/build.sh -l')
os.system('bash build/astra_analytical/build.sh')

### Workload Generation

In [ ]:
os.chdir(chakra_dir)
os.system('rm *.et')
os.system('python3 -m utils.et_generator.et_generator --num_npus 64 --num_dims 1 --default_comm_size 16384')

## Collective Impl Generation

### MSCCLang -> XML

In [ ]:
os.chdir(mscclang_dir)
os.system(f'python3 allreduce_a100_ring.py 64 1 1 > {demodir}/demo_allreduce.xml')

### XML -> Chakra

In [ ]:
os.chdir(demodir)
os.system('rm *.et')

os.chdir(converter_dir)
collective_size = int(16 * 1024) # 16 Kilobytes
os.system(f'python3 -m et_converter.et_converter \
        --input_type        msccl \
        --input_filename    {demodir}/demo_allreduce.xml \
        --output_filename   {demodir}/allreduce \
        --num_dims          1 \
        --coll_size         {collective_size}')

## Running Stuff

### AllReduce using traditional Ring Impl

In [ ]:
os.chdir(astra_sim_dir)

# Common Config
SYSTEM_CONFIG="./inputs/system/Ring.json"
MEMORY_CONFIG="./inputs/remote_memory/analytical/no_memory_expansion.json"
WORKLOAD_CONFIG="./extern/graph_frontend/chakra/one_comm_coll_node_allreduce"
NETWORK_CONFIG="./inputs/network/analytical/Ring.yml"

# Run
os.system(f'{astra_sim_dir}/build/astra_analytical/build/bin/AstraSim_Analytical_Congestion_Unaware \
    --workload-configuration={WORKLOAD_CONFIG} \
    --system-configuration={SYSTEM_CONFIG} \
    --network-configuration={NETWORK_CONFIG} \
    --remote-memory-configuration={MEMORY_CONFIG}')


### Using Chakra Impl as the "Workload" Input

In [ ]:
os.chdir(astra_sim_dir)

# Common Config
SYSTEM_CONFIG="./inputs/system/Ring.json"
MEMORY_CONFIG="./inputs/remote_memory/analytical/no_memory_expansion.json"
WORKLOAD_CONFIG=f"./demo/allreduce"
NETWORK_CONFIG="./inputs/network/analytical/Ring.yml"
      # Run
os.system(f'{astra_sim_dir}/build/astra_analytical/build/bin/AstraSim_Analytical_Congestion_Unaware \
    --workload-configuration={WORKLOAD_CONFIG} \
    --system-configuration={SYSTEM_CONFIG} \
    --network-configuration={NETWORK_CONFIG} \
    --remote-memory-configuration={MEMORY_CONFIG}')


### Using both Workload input & Comm API Input

In [ ]:
os.chdir(astra_sim_dir)

# Common Config
SYSTEM_CONFIG="./inputs/system/ChakraImpl.json"
MEMORY_CONFIG="./inputs/remote_memory/analytical/no_memory_expansion.json"
WORKLOAD_CONFIG="./extern/graph_frontend/chakra/one_comm_coll_node_allreduce"
NETWORK_CONFIG="./inputs/network/analytical/Ring.yml"

os.system(f'{astra_sim_dir}/build/astra_analytical/build/bin/AstraSim_Analytical_Congestion_Unaware \
    --workload-configuration={WORKLOAD_CONFIG} \
    --system-configuration={SYSTEM_CONFIG} \
    --network-configuration={NETWORK_CONFIG} \
    --remote-memory-configuration={MEMORY_CONFIG}')
